<a href="https://colab.research.google.com/github/samarreguigui/AI_Ethik/blob/main/prompttransformations_summeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#defining the evaluation scope
# 5 Scopes from the JUDGE-BENCH

In [ ]:
!pip install -U openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


In [ ]:
import json
from openai import OpenAI
import os
import re


In [ ]:
#os.environ["OPENAI_API_KEY"] = Key

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
def judge(prompt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
        max_tokens=5
    )
    return response.choices[0].message.content.strip()



In [ ]:

#Save experience results
def save_results(results, filename):
    """
    Save experiment results to a JSON file.

    Args:
        results (list or dict): data to save
        filename (str): base filename, e.g. "summeval_coherence_P0_P1_raw_results.json"
        add_timestamp (bool): if True, appends a timestamp to the filename
    """

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print("Saved:", filename)
    return filename


In [ ]:
summeval_path = "/content/summeval.json"

with open(summeval_path, "r", encoding="utf-8") as f:
    summeval_raw = json.load(f)

len(summeval_raw)

7

In [ ]:
#Load the JSON and extract the coherence prompt template
with open("summeval.json", "r", encoding="utf-8") as f:
    schema = json.load(f)

coherence_annotation = next(
    a for a in schema["annotations"]
    if a["metric"] == "coherence"
)

base_prompt = coherence_annotation["prompt"]
min_score = coherence_annotation["worst"]
max_score = coherence_annotation["best"]

print("Loaded", schema["dataset"])
print("Scale", min_score, "to", max_score)
print("Instances", len(schema["instances"]))


Loaded SummEval (Fabbri et al., Transactions of the Association for Computational Linguistics 2021)
Scale 1 to 5
Instances 1600


In [ ]:
print(base_prompt)

You will be given one summary written for a news article.

Your task is to rate the summary on one metric.

Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby "the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic."

Evaluation Steps:

1. Read the news article carefully and identify the main topic and key points.
2. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is 

In [ ]:
#Perform Perturbation
P1_template = """
You will be given one summary written for a news article.

Your task is to rate the summary on one metric.

Evaluation Criteria:

Coherence (1 to 5) is the overall quality of how well the sentences fit together. A coherent summary is well organized and reads as a single unified text rather than a loose collection of statements.

Evaluation Steps:

1. Read the article and identify its main topic and key points.
2. Read the summary and compare it to the article.
3. Give a coherence score from 1 to 5.

{{ instance }}

Evaluation Form, scores only.

Coherence:
"""



In [ ]:
#Runit on 20 instance and save it

def parse_score(text: str):
    match = re.search(r"\d+", text)
    if not match:
        return None
    return int(match.group())


def run_summeval_prompt(
    schema,
    prompt_template,
    prompt_name,
    judge_fn,
    output_filename,
    n_instances=20
):
    """
    Run a single prompt on SummEval instances and save results.

    Args:
        schema (dict): loaded summeval JSON (with "instances")
        prompt_template (str): prompt containing {{ instance }}
        prompt_name (str): label, e.g. "P0", "P1", "P_combined"
        judge_fn (callable): function(prompt) -> raw model output
        output_filename (str): where to save JSON
        n_instances (int): number of instances to evaluate
    """

    results = []

    subset = schema["instances"][:n_instances]

    for item in subset:
        instance_id = item["id"]
        instance_text = item["instance"]

        prompt = prompt_template.replace("{{ instance }}", instance_text)
        raw_output = judge_fn(prompt)
        parsed = parse_score(raw_output)

        results.append({
            "instance_id": instance_id,
            "prompt": prompt_name,
            "prompt_text": prompt,
            "raw_output": raw_output,
            "parsed_score": parsed
        })

    save_results(results, output_filename)
    return results

results_P0 = run_summeval_prompt(
    schema=schema,
    prompt_template=base_prompt,
    prompt_name="P0",
    judge_fn=judge,
    output_filename="summeval_coherence_P0_raw_results.json",
    n_instances=20
)
results_P1 = run_summeval_prompt(
    schema=schema,
    prompt_template=P1_template,
    prompt_name="P1",
    judge_fn=judge,
    output_filename="summeval_coherence_P1_raw_results.json",
    n_instances=20
)






Saved: summeval_coherence_P0_raw_results.json
Saved: summeval_coherence_P1_raw_results.json
Saved: summeval_coherence_P1_raw_results.json
Saved: summeval_coherence_P0_raw_results.json


'summeval_coherence_P0_raw_results.json'

"You will be given one summary written for a news article.\n\nYour task is to rate the summary on one metric.\n\nPlease make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n\nEvaluation Criteria:\n\nCoherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby \"the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic.\"\n\nEvaluation Steps:\n\n1. Read the news article carefully and identify the main topic and key points.\n2. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.\n3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.{{ instance }}\n\n\nEvaluation Form (scores ONLY):\n\n- Coherence:",
            "worst": 1,
            "best": 5
       


In [ ]:
##Perform Perturbation P2

'''Changes:

Instance moved to the top
(1-5) rewritten as (1 to 5)
Synonym substitution
Replaced 1,2,3 with words first then next
Synonym substitution
“rate” → “assess”
“collective quality” → “overall structural quality”
“heap of related information” → “collection of related statements”
“assign a score” → “provide … score”
'''

p2_template="""{{ instance }} You will be given one summary written for a news article.\n\nYour task is to assess the summary on one metric.
\n\nPlease make sure you read and understand these instructions carefully.
Please keep this document open while reviewing, and refer to it as needed.\n\nEvaluation Criteria:\n\nCoherence (1-5) - the overall structural quality of all sentences. We align this dimension with the DUC quality question
 of structure and coherence whereby "the summary should be well-structured and well-organized.
  The summary should not just be a collection of related statements, but should build from sentence to a coherent body of information about a topic.
  "\n\nEvaluation
  \n\nFirst. Read the news article carefully and identify the main topic and key points.\nNext. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it presents them in a clear and logical order.\nThen. Provide a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.\n\n\nEvaluation Form (scores ONLY):\n\n- Coherence:", "worst": 1, "best": 5"""

results_P2 = run_summeval_prompt(
    schema=schema,
    prompt_template=p2_template,
    prompt_name="P2",
    judge_fn=judge,
    output_filename="summeval_coherence_P2_raw_results.json",
    n_instances=20
)


Saved: summeval_coherence_P2_raw_results.json


# **Comparing the results**

In [ ]:
import pandas as pd


In [ ]:
#oad the three result files
def load_results(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)
p0 = load_results("summeval_coherence_P0_raw_results.json")
p1 = load_results("summeval_coherence_P1_raw_results.json")
p2 = load_results("summeval_coherence_P2_raw_results.json")
#Step 2. Index by instance_id
def index_by_id(results):
    return {r["instance_id"]: r["parsed_score"] for r in results}

p0_scores = index_by_id(p0)
p1_scores = index_by_id(p1)
p2_scores = index_by_id(p2)


#Compare and build the table
comparison = []

for instance_id in p0_scores:
    score_p0 = p0_scores.get(instance_id)
    score_p1 = p1_scores.get(instance_id)
    score_p2 = p2_scores.get(instance_id)

    comparison.append({
        "instance_id": instance_id,
        "same_P1": score_p1 == score_p0,
        "same_P2": score_p2 == score_p0
    })

#Display as a table
for row in comparison:
    print(row)


{'instance_id': 0, 'same_P1': True, 'same_P2': True}
{'instance_id': 1, 'same_P1': False, 'same_P2': False}
{'instance_id': 2, 'same_P1': True, 'same_P2': True}
{'instance_id': 3, 'same_P1': True, 'same_P2': False}
{'instance_id': 4, 'same_P1': True, 'same_P2': True}
{'instance_id': 5, 'same_P1': True, 'same_P2': True}
{'instance_id': 6, 'same_P1': True, 'same_P2': True}
{'instance_id': 7, 'same_P1': True, 'same_P2': False}
{'instance_id': 8, 'same_P1': True, 'same_P2': False}
{'instance_id': 9, 'same_P1': True, 'same_P2': False}
{'instance_id': 10, 'same_P1': True, 'same_P2': True}
{'instance_id': 11, 'same_P1': True, 'same_P2': True}
{'instance_id': 12, 'same_P1': True, 'same_P2': False}
{'instance_id': 13, 'same_P1': True, 'same_P2': True}
{'instance_id': 14, 'same_P1': True, 'same_P2': False}
{'instance_id': 15, 'same_P1': True, 'same_P2': False}
{'instance_id': 16, 'same_P1': True, 'same_P2': True}
{'instance_id': 17, 'same_P1': True, 'same_P2': False}
{'instance_id': 18, 'same_P1

#Focusing on the variation
# keep only instances with any variation


In [ ]:
comparison_values = []

for instance_id in p0_scores:
    score_p0 = p0_scores.get(instance_id)
    score_p1 = p1_scores.get(instance_id)
    score_p2 = p2_scores.get(instance_id)

    comparison_values.append({
        "instance_id": instance_id,
        "P0": score_p0,
        "P1": score_p1,
        "P2": score_p2
    })
filtered = [
    row for row in comparison_values
    if row["P1"] != row["P0"] or row["P2"] != row["P0"]
]
df_filtered = pd.DataFrame(filtered)
df_filtered

,instance_id,P0,P1,P2
0,1,2,3,3
1,3,2,2,3
2,7,3,3,4
3,8,1,1,3
4,9,3,3,4
5,12,3,3,4
6,14,3,3,4
7,15,2,2,3
8,17,3,3,4
9,18,3,4,4


I notice that p3 tend to give higher evaluations. try to understand the reason:

main difference between base prompt and p2 :
Difference

P0: {{ instance }} appears at the end, after criteria and steps

P2: {{ instance }} appears at the very beginning

The model forms an initial semantic impression of the text, then later encounters evaluation criteria. That initial impression biases the final score upward.

In P0, the model is primed as a judge first, reader second.

Effect from using synonym

You removed strongly negative metaphors.

“Heap” implies disorder and failure.
“Collection” is neutral.


**Compare to human evaluation see if the new prompt tent to be more similar to human or not**

In [ ]:
def extract_human_coherence(summeval_raw):
    human = {}

    for item in summeval_raw["instances"]:
        instance_id = item["id"]
        score = item["annotations"]["coherence"]["mean_human"]
        human[instance_id] = score

    return human


In [ ]:
human_scores = extract_human_coherence(summeval_raw)


In [ ]:
# ================================
# FULL PIPELINE: HUMAN vs P0 / P1 / P2 (FIRST 20 INSTANCES)
# ================================

import json
import pandas as pd
from scipy.stats import spearmanr

# ----------------
# Step 0. Load SummEval (JUDGE-BENCH converted) JSON
# ----------------
with open("summeval.json", "r", encoding="utf-8") as f:
    summeval_raw = json.load(f)

# ----------------
# Step 1. Extract human coherence scores
# ----------------
def extract_human_coherence(summeval_raw):
    human = {}
    for item in summeval_raw["instances"]:
        instance_id = item["id"]
        score = round(item["annotations"]["coherence"]["mean_human"])
        human[instance_id] = score
    return human

human_scores = extract_human_coherence(summeval_raw)

# ----------------
# Step 2. Fix the evaluation subset (first 20 instances)
# ----------------
INSTANCE_IDS = list(range(20))

human_20 = {
    iid: human_scores[iid]
    for iid in INSTANCE_IDS
    if iid in human_scores
}

assert len(human_20) == 20, "Human subset must contain exactly 20 instances"

# ----------------
# Step 3. Helper to extract model scores per prompt
# ----------------
def extract_model_scores(results, instance_ids):
    scores = {}
    for r in results:
        iid = r["instance_id"]
        if iid in instance_ids and r["parsed_score"] is not None:
            scores[iid] = r["parsed_score"]
    return scores

# EXPECTED: results_P0, results_P1, results_P2 already loaded
model_P0 = extract_model_scores(results_P0, INSTANCE_IDS)
model_P1 = extract_model_scores(results_P1, INSTANCE_IDS)
model_P2 = extract_model_scores(results_P2, INSTANCE_IDS)

# ----------------
# Step 4. Align vectors for correlation
# ----------------
def aligned_vectors(human, model, instance_ids):
    h, m = [], []
    for iid in instance_ids:
        if iid in human and iid in model:
            h.append(human[iid])
            m.append(model[iid])
    return h, m

h0, m0 = aligned_vectors(human_20, model_P0, INSTANCE_IDS)
h1, m1 = aligned_vectors(human_20, model_P1, INSTANCE_IDS)
h2, m2 = aligned_vectors(human_20, model_P2, INSTANCE_IDS)

# ----------------
# Step 5. Compute Spearman correlations
# ----------------
corr_P0 = spearmanr(h0, m0).correlation
corr_P1 = spearmanr(h1, m1).correlation
corr_P2 = spearmanr(h2, m2).correlation

print("Spearman correlations:")
print("P0 vs Human:", corr_P0)
print("P1 vs Human:", corr_P1)
print("P2 vs Human:", corr_P2)

# ----------------
# Step 6. Build per-instance comparison table
# ----------------
rows = []
for iid in INSTANCE_IDS:
    rows.append({
        "instance_id": iid,
        "human_score": human_20.get(iid),
        "P0_score": model_P0.get(iid),
        "P1_score": model_P1.get(iid),
        "P2_score": model_P2.get(iid),
    })

df = pd.DataFrame(rows)


# ----------------
# Step 7. Save results (important for Colab)
# ----------------
output_path = "summeval_first20_human_vs_P0_P1_P2.csv"
df.to_csv(output_path, index=False)

print("Saved:", output_path)

# ----------------
# Step 8. Display table
# ----------------
df


Spearman correlations:
P0 vs Human: 0.3784327829053303
P1 vs Human: 0.3699602523853969
P2 vs Human: 0.5217908041195821
Saved: summeval_first20_human_vs_P0_P1_P2.csv


,instance_id,human_score,P0_score,P1_score,P2_score
0,0,1,2,2,2
1,1,2,2,3,3
2,2,2,3,3,3
3,3,2,2,2,3
4,4,3,3,3,3
5,5,4,3,3,3
6,6,3,3,3,3
7,7,3,3,3,4
8,8,5,1,1,3
9,9,4,3,3,4


Although prompt perturbations increase scores, they do not consistently improve alignment with human judgments, suggesting that higher scores reflect softened evaluation standards rather than more accurate assessment.